In [1]:
import pyspark as ps
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import os
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
# os.environ["PYSPARK_PYTHON"] = "/home/serhii/anaconda3/bin/python"

In [2]:
file = "data/ukwiki_0.csv"

spark = SparkSession.builder.master("local").appName("WikiParse").getOrCreate()

df = spark.read.csv(file, header=True, sep=',')

In [3]:
stop_words = []

with open('data/stop_words') as file:
    for line in file:
        stop_words.append(line.strip())

In [4]:
# stop_words

In [5]:
df_trip = df.select(['Title', 'Text'])\
    .withColumn('Text', regexp_replace('Text', '[§»«·&\~.a-zA-Z^=\-\"<>!?:;{}()\[\]/|%0-9\\\+\*#_]+', ' '))\
    .withColumn('Text', regexp_replace('Text', '\'{3}', ' '))\
    .withColumn('Text', regexp_replace('Text', '[—−]', ' '))\
    .withColumn('Text', regexp_replace('Text', '[^а-яА-ЯіІіІєЄҐґїЇ\s]', ''))\
    .withColumn('Text', regexp_replace('Text', '\s+', ' '))\
    .select([trim(lower(col('Title'))).alias('Title'), trim(lower(col('Text'))).alias('Text')])\

#     .withColumn("Text", split("Text", "\s+"))
#     .withColumn('Text', regexp_replace('Text', '[òо́]', 'о'))\
#     .withColumn('Text', regexp_replace('Text', 'а́', 'а'))\
#     .withColumn('Text', regexp_replace('Text', 'í', 'і'))\
#     .withColumn('Text', regexp_replace('Text', 'и́', 'и'))\
#     .withColumn('Text', regexp_replace('Text', '[е́é]', 'е'))\
#     .withColumn('Text', regexp_replace('Text', '[́м]', 'м'))\
#     .withColumn('Text', regexp_replace('Text', '\'', ''))\
#     .withColumn('Text', regexp_replace('Text', '[α-ωΑ-Ωºäüöñ¹²³⁴]+', ''))\
#     .withColumn('Text', regexp_replace('Text', '\s+', ' '))\
#     .select([trim(lower(col('Title'))), trim(lower(col('Text')))])

In [6]:
from pyspark.ml.feature import Tokenizer

In [7]:
tokenizer = Tokenizer(inputCol="Text", outputCol="Vector")

vector_df = tokenizer.transform(df_trip).select("vector")

In [8]:
vector_df.show(10)

+--------------------+
|              vector|
+--------------------+
|[шапка, головна, ...|
|[файл, фізична, к...|
|[атом, значення, ...|
|[мільярд, число, ...|
|[ядро, основна, ч...|
|[файл, пкс, римсь...|
|[файл, хімічні, р...|
|[файл, коло, з, ц...|
|[файл, міжнародни...|
|[біологія, життя,...|
+--------------------+
only showing top 10 rows



In [9]:
from pyspark.ml.feature import StopWordsRemover

In [10]:
remover = StopWordsRemover(inputCol="vector", outputCol="vector_no_stopwords" ,stopWords=stop_words)

stopwords = remover.getStopWords() 

stopwords[:10]

['a', 'б', 'в', 'г', 'е', 'ж', 'з', 'м', 'т', 'у']

In [11]:
vector_no_stopw_df = remover.transform(vector_df).select("vector_no_stopwords")

In [12]:
vector_no_stopw_df.show(10)

+--------------------+
| vector_no_stopwords|
+--------------------+
|[шапка, головна, ...|
|[файл, фізична, к...|
|[атом, значення, ...|
|[мільярд, число, ...|
|[ядро, основна, ч...|
|[файл, пкс, римсь...|
|[файл, хімічні, р...|
|[файл, коло, цент...|
|[файл, міжнародни...|
|[біологія, слово,...|
+--------------------+
only showing top 10 rows



In [13]:
import re

class UkrainianStemmer():
    def __init__(self, word):
        self.word = word
        self.vowel = r'аеиоуюяіїє'  # http://uk.wikipedia.org/wiki/Голосний_звук
        self.perfectiveground = r'(ив|ивши|ившись|ыв|ывши|ывшись((?<=[ая])(в|вши|вшись)))$'
        # http://uk.wikipedia.org/wiki/Рефлексивне_дієслово
        self.reflexive = r'(с[яьи])$'
        # http://uk.wikipedia.org/wiki/Прикметник + http://wapedia.mobi/uk/Прикметник
        self.adjective = r'(ими|ій|ий|а|е|ова|ове|ів|є|їй|єє|еє|я|ім|ем|им|ім|их|іх|ою|йми|іми|у|ю|ого|ому|ої)$'
        # http://uk.wikipedia.org/wiki/Дієприкметник
        self.participle = r'(ий|ого|ому|им|ім|а|ій|у|ою|ій|і|их|йми|их)$'
        # http://uk.wikipedia.org/wiki/Дієслово
        self.verb = r'(сь|ся|ив|ать|ять|у|ю|ав|али|учи|ячи|вши|ши|е|ме|ати|яти|є)$'
        # http://uk.wikipedia.org/wiki/Іменник
        self.noun = r'(а|ев|ов|е|ями|ами|еи|и|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я|і|ові|ї|ею|єю|ою|є|еві|ем|єм|ів|їв|ю)$'
        self.rvre = r'[аеиоуюяіїє]'
        self.derivational = r'[^аеиоуюяіїє][аеиоуюяіїє]+[^аеиоуюяіїє]+[аеиоуюяіїє].*(?<=о)сть?$'
        self.RV = ''

    def ukstemmer_search_preprocess(self, word):
        word = word.lower()
        word = word.replace("'", "")
        word = word.replace("ё", "е")
        word = word.replace("ъ", "ї")
        return word

    def s(self, st, reg, to):
        orig = st
        self.RV = re.sub(reg, to, st)
        return (orig != self.RV)

    def stem_word(self):
        word = self.ukstemmer_search_preprocess(self.word)
        if not re.search('[аеиоуюяіїє]', word):
            stem = word
        else:
            p = re.search(self.rvre, word)
            start = word[0:p.span()[1]]
            self.RV = word[p.span()[1]:]

            # Step 1
            if not self.s(self.RV, self.perfectiveground, ''):

                self.s(self.RV, self.reflexive, '')
                if self.s(self.RV, self.adjective, ''):
                    self.s(self.RV, self.participle, '')
                else:
                    if not self.s(self.RV, self.verb, ''):
                        self.s(self.RV, self.noun, '')
            # Step 2
            self.s(self.RV, 'и$', '')

            # Step 3
            if re.search(self.derivational, self.RV):
                self.s(self.RV, 'ость$', '')

            # Step 4
            if self.s(self.RV, 'ь$', ''):
                self.s(self.RV, 'ейше?$', '')
                self.s(self.RV, 'нн$', u'н')

            stem = start + self.RV
        return stem

In [14]:
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = UkrainianStemmer(t).stem_word()
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

from pyspark.sql.types import *

stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

In [15]:
vector_stemmed_df = (
    vector_no_stopw_df
        .withColumn("vector_stemmed", stemmer_udf("vector_no_stopwords"))
        .select("vector_stemmed")
  )

In [16]:
def list_to_str(in_vec):
    return " ".join([w for w in in_vec])

list_to_str_udf = udf(lambda x: list_to_str(x), StringType())

In [17]:
result = vector_stemmed_df.withColumn('Text', list_to_str_udf('vector_stemmed'))

In [18]:
final = result.toPandas()

In [19]:
final['Text'].to_csv('preproc_0.csv')

In [20]:
vector_stemmed_df

DataFrame[vector_stemmed: array<string>]

In [21]:
from pyspark.ml.feature import Word2Vec

In [31]:
word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol='vector_stemmed', outputCol='model')

In [32]:
model = word2Vec.fit(vector_stemmed_df)

In [155]:
vectors = model.getVectors()
size = 100
amount = vectors.count()

vectors_collection = vectors.collect()

f = open('custom.embeddings', 'w')

f.write('{} {}\n'.format(amount, size))

for row in vectors_collection:
    vec_str = ' '.join([ str(np.round(x, 5)) for x in row['vector']])
    f.write(row['word'] + ' ' + vec_str + '\n')
    
f.close()

In [159]:
def save_model(w2v_model, file_name):
    vectors = w2v_model.getVectors()
    size = len(vectors.first()['vector'])
    amount = vectors.count()

    vectors_collection = vectors.collect()

    f = open(file_name, 'w')

    f.write('{} {}\n'.format(amount, size))

    for row in vectors_collection:
        vec_str = ' '.join([ str(np.round(x, 5)) for x in row['vector']])
        f.write(row['word'] + ' ' + vec_str + '\n')

    f.close()
    
save_model(model, 'custom.embeddings')

In [152]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim

In [153]:
gmodel = gensim.models.KeyedVectors.load_word2vec_format("custom.embeddings")

/home/serhii/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [154]:
gmodel.similar_by_vector(gmodel['київ'])

[('київ', 1.0),
 ('всеукр', 0.8452724814414978),
 ('шубравськ', 0.8207768201828003),
 ('бібліотечк', 0.8139780163764954),
 ('шевченкознавч', 0.8110851049423218),
 ('петраш', 0.8075677752494812),
 ('печатн', 0.8074122071266174),
 ('соціол', 0.7977311611175537),
 ('пядесят', 0.7971876859664917),
 ('одмін', 0.796441912651062)]